In [1]:
import matplotlib
from src.repository.surebet_repository import SurebetRepository
from src.repository.game_repository import GameRepository
from src.genetic.arbitrage import CrossOverEnum, Population
from src.util import ExportadorDeGraficos, PickleSerializer
from datetime import datetime
game_repository = GameRepository()
repository = SurebetRepository()

<h3> Parâmetros para execução do experimento </h3>

In [10]:
n_generations = 200
n_individuals = 30
mutation_rate = 0.05
crossover_rate = 0.98
crossover=CrossOverEnum.ONE_POINT_CROSSOVER

<h3>Métodos auxiliares para plot, consulta ao banco e munipulação dos dados</h3>

In [3]:
def get_teams(ids):
    return game_repository.get_teams(ids)

In [4]:
def paretos_frontier_plot(front, selected_individuals, path, best_frontier):
    plt = matplotlib.pyplot
    plt.figure()
    plt.title(f'Pareto Front')
    plt.xlabel('Profit')
    plt.ylabel('Dispersation')
    plt.scatter([i.fitness_normalized[0] for i in front], [i.fitness_normalized[1] for i in front])
    plt.scatter([i.fitness_normalized[0] for i in selected_individuals], [i.fitness_normalized[1] for i in selected_individuals], color='#2BF063')
    plt.plot([i.fitness_normalized[0] for i in front], [i.fitness_normalized[1] for i in front], color='red', linewidth=1, linestyle='-')
    plt.savefig(path)
    plt.close()


def pick_up_individuals(front):   
    front_fitness = list({tuple(i.fitness) for i in front})  # Remove os itens repetidos
    OPTIMIZE_PROFIT = -1
    OPTIMIZE_MIDDLE = ((len(front_fitness) // 2) - 1)
    OPTIMIZE_DISPERSATION = 0
    
    front_sorted = sorted(front, key=lambda x: x.fitness[0])
    front_fitness = sorted(front_fitness, key=lambda x: x[0])
    
    aux_individuals_selected = front_fitness[OPTIMIZE_PROFIT], front_fitness[OPTIMIZE_MIDDLE], front_fitness[OPTIMIZE_DISPERSATION]  # Seleciona os 3 indivíduos (Aqui só tem os fitness)
    selected = []
    
    # Método para descobrir qual a é o indivíduo(dentro da fronteira de pareto que possui vários objetos) que possui o fitness escolhido anteriormente
    for i in aux_individuals_selected:
        for individual in front_sorted:
            if list(i) == individual.fitness:
                selected.append(individual)
                break

    return front_sorted, selected

def normalize(front, high_fitness_evaluated):
    max_profit = high_fitness_evaluated[0]
    max_dispersation = high_fitness_evaluated[1]
    for i in front:
        profit_normalized = i.fitness[0] / max_profit
        dispersation_normalized = i.fitness[1] / max_dispersation
        i.fitness_normalized = [profit_normalized, dispersation_normalized]
    

In [5]:
def export_frontier(export: ExportadorDeGraficos, population: Population):
    generation_plot = [0, population.n_generations - 1, population.best_frontier]
    for generation, front in population.pareto_history_front_normalized:
        x = [i[0] for i in front]
        y = [i[1] for i in front]
        if generation in generation_plot:
            export.plot(x, y, generation+1)

def export_solution(population: Population, front_sorted, individuals_selected, export: ExportadorDeGraficos):
    export.hypervolume_plot(population)
    PickleSerializer.save(population, f'{export.data_hora_atual}/pickle.pkl')
    path = export.data_hora_atual + '/' + export.get_file_name(population.best_frontier, 'png')
    export_frontier(export, population)
    paretos_frontier_plot(front_sorted, individuals_selected, path, population.best_frontier)
    
    
def plot_frontier(population: Population, export):
    frontier = population.pareto_history_front[population.best_frontier][1]
    front_sorted, individuals_selected = pick_up_individuals(frontier)
    
    normalize(front_sorted, population.high_fitness_evaluated)
    normalize(individuals_selected, population.high_fitness_evaluated)
    
    export_solution(population, front_sorted, individuals_selected, export)
    
    return individuals_selected

In [6]:
from src.genetic.arbitrage import Individual

def decode_individual(individual: Individual, population: Population, individual_label, report_header, path, teams_dict: dict, verbose=True):
    profit = individual.fitness[0]
    number_of_surebets_chosen = sum([i[0] for i in individual.chromosome])
    surebets_selected = []
        
    for gene, arbitrage in zip(individual.chromosome, individual.arbitrages):
        if gene[0] == 1:
            surebets_selected.append((f'Game: {teams_dict.get(arbitrage.game_id)}', f'Bookmaker 1: {arbitrage.bookmaker_key_OVER}', f'Bookmaker 2: {arbitrage.bookmaker_key_UNDER}', f'Amount bet R$ {round(gene[1] * population.budget, 2)}', f'Profit R$ {((gene[1]*population.budget)*(arbitrage.profit/100)):,.2f}', f'{arbitrage.profit:,.2f}%'))
    
    if verbose:
        print(individual_label)
        print(f'Number of surebets chosen: {number_of_surebets_chosen}')
        print('Surebets Selected:')
        for i in surebets_selected:
            print(i)
    
        print(f'Total Profit: R$ {profit:,.2f}')
        print('-------------------------------------------------------------------------------------------------------------------------------------------------------')
    
    with open(path, 'a') as file:
        if report_header is not None:
            file.write(report_header)

        file.write(f'{individual_label}\n')
        file.write(f'Number of surebets chosen: {number_of_surebets_chosen}\n')
        file.write('Surebets Selected:\n')
        for i in surebets_selected:
            file.write(str(i)+'\n')
        file.write(f'Total Profit: R$ {profit:,.2f}\n')
        file.write('-------------------------------------------------------------------------------------------------------------------------------------------------------\n')
        
    

In [7]:
def run_scenario(start_date = '2023-09-02 14:00:00', end_date = '2023-09-02 14:05:00', verbose=True, count=0):
    export = ExportadorDeGraficos()
    
    arbitrages = repository.find_all_unique_between(start_date, end_date)
    population = Population(n_generations=n_generations, n_individuals=n_individuals, mutation_rate=mutation_rate, crossover_rate=crossover_rate, 
                            budget=1000, crossover_strategy=crossover, arbitrages=arbitrages)
    
    teams_dict = get_teams([i.game_id for i in arbitrages])

    individuals_picked_up = plot_frontier(population, export)
    
    labels = ['Profit Optimization', 'Equilibrium Optimization', 'Dispersation Optimization']
    folder = export.data_hora_atual
    start = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S').strftime('%H:%M')
    end = datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S').strftime('%H:%M')
    date_event = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S').strftime('%d/%m/%Y')
    path = f"{folder}/report_{start}_{end}.txt"
    
    report_header = f"""************************************************************* EXPERIMENTAL RESULT REPORT *************************************************************

[Date]: {date_event}
[Surebet Interval]: {start} to {end}
[Best frontier]: {population.best_frontier + 1}º generation
[Budget]: R$ {population.budget:,.2f}

"""
    for label, individual in zip(labels, individuals_picked_up):
        decode_individual(individual, population, label, report_header, path, teams_dict, verbose)
        report_header = None


<h3> Cenários mês de setembro </h3>
<img src="images/surebets_setembro.png" width="75%">
<br> <br>
<img src="images/surebets_02_setembro.png" width="75%">

<h3> Cenário 02/09/2023 </h3>

In [11]:
run_scenario(start_date='2023-09-02 14:00:00', end_date='2023-09-02 14:05:00', verbose=False)

1º
Index: 175 [TAM]: 31
2º
Index: 143 [TAM]: 2
3º
Index: 57 [TAM]: 32
4º
Index: 135 [TAM]: 4
5º
Index: 114 [TAM]: 3
6º
Index: 104 [TAM]: 3
7º
Index: 181 [TAM]: 31
8º
Index: 0 [TAM]: 5
9º
Index: 43 [TAM]: 32
10º
Index: 192 [TAM]: 29
11º
Index: 127 [TAM]: 17
12º
Index: 170 [TAM]: 18
13º
Index: 191 [TAM]: 28
14º
Index: 153 [TAM]: 30
15º
Index: 140 [TAM]: 3
16º
Index: 199 [TAM]: 30
17º
Index: 195 [TAM]: 15
18º
Index: 35 [TAM]: 31
19º
Index: 81 [TAM]: 4
20º
Index: 5 [TAM]: 30



### Cenário 24/09/2023
